<a href="https://colab.research.google.com/github/fabriciobotelho99/odyssey-project/blob/main/processing%20mamoas/training_yolov5_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo

shutil.copytree("/content/yolov5", "/content/drive/MyDrive/yolov5") #copy repro to drive

%cd drive/MyDrive/yolov5

%pip install -qr requirements.txt # install dependencies

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/MyDrive/yolov5
     |████████████████████████████████| 596 kB 7.5 MB/s 


In [ ]:
%cd drive/MyDrive/yolov5
%pip install -qr requirements.txt # install dependencies

/content/drive/MyDrive/yolov5
     |████████████████████████████████| 596 kB 29.9 MB/s 


In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images
import glob
import cv2
import os.path
from pathlib import Path
import shutil
from os.path import exists
import yaml

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla T4)


In [ ]:
n_epochs = 200
batch_size = 16
IoU_t = 0.2
k_fold = 6
nc = 1
names = 'mamoa'

In [ ]:
def createYOLOYaml(t_dir, v_dir, n_classes, names):
  mamoas = dict(
    train = t_dir,
    val = v_dir,
    nc = n_classes)
  
  with open('/content/drive/MyDrive/yolov5/data/mamoas.yaml', 'w') as f:
      data = yaml.dump(mamoas, f, default_flow_style=False)

In [ ]:
def splitTrainData(n_folds, train_dir, fold_dir):
  # read files into train directory
  if(os.path.exists(train_dir)):
    files = os.listdir(train_dir)
    files.sort()
    if(os.path.exists(fold_dir) == False):
      os.makedirs(fold_dir)
      print("Path: " + fold_dir + " doesn't exist.")
      print("Created " + fold_dir)
    else:
      shutil.rmtree(fold_dir)
      os.makedirs(fold_dir)
      print(fold_dir + " restaured.")
    flag = int(len(files)/n_folds)
    k, y = 0, flag
    for j in range(1, n_folds+1):
      os.makedirs(fold_dir + "fold" + str(j) + "/")
      for i in range(k, y):
        shutil.copy(train_dir + files[i], fold_dir + "fold" + str(j) + "/" + files[i])
      k = y
      y += flag
  else:
    print("Path: " + train_dir + " doesn't exist")

  fold_dirs=[]
  for path in os.listdir(fold_dir):
     fold_dirs.append(os.path.join(fold_dir, path))

  return fold_dirs

In [ ]:
def crossValidation(n_folds, fold, train_dir, fold_val, fold_dir):
  print("Using " + str(n_folds) + " folds.")
  if(os.path.exists(train_dir)): # update train directory
    shutil.rmtree(train_dir)
    for j in range(1, n_folds+1):
      if(j != fold+1):
        if(os.path.exists(train_dir)):
          src_files = os.listdir(fold_dir +"fold" + str(j))
          for file_name in src_files:
            full_file_name = os.path.join(fold_dir +"fold" + str(j), file_name)
            shutil.copy(full_file_name, train_dir)
        else:
          shutil.copytree(fold_dir +"fold" + str(j), train_dir)
      else:
        print("Using fold number:", fold+1)

  return train_dir, fold_val[fold]

In [ ]:
# calculate mean of final results of cross validation
def mean_results(save_results, k_fold):
  precision, recall, mAP05, mAP05_095, box_loss, obj_loss, cls_loss = [],[],[],[],[],[],[]

  for i in range(1, k_fold+1):
    if(i==1): # read exp 0
      if(os.path.exists(save_results + "exp/")): 
        f = open(save_results + "exp/result.txt",'r')
        content = f.read()
        print(content)
        content_list = content.replace('(', '').replace(')', '').split(',')
        precision.append(float(content_list[0]))
        recall.append(float(content_list[1]))
        mAP05.append(float(content_list[2]))
        mAP05_095.append(float(content_list[3]))
        box_loss.append(float(content_list[4]))
        obj_loss.append(float(content_list[5]))
        cls_loss.append(float(content_list[6]))
        f.close()
    else:
      if(os.path.exists(save_results + "exp" + str(i) + "/")): 
        f = open(save_results + "exp" + str(i) + "/result.txt",'r')
        content = f.read()
        print(content)
        content_list = content.replace('(', '').replace(')', '').split(',')
        precision.append(float(content_list[0]))
        recall.append(float(content_list[1]))
        mAP05.append(float(content_list[2]))
        mAP05_095.append(float(content_list[3]))
        box_loss.append(float(content_list[4]))
        obj_loss.append(float(content_list[5]))
        cls_loss.append(float(content_list[6]))
        f.close()
  
  p = sum(precision)/k_fold
  r = sum(recall)/k_fold
  mAP5 = sum(mAP05)/k_fold
  mAP5_95 = sum(mAP05_095)/k_fold
  bloss = sum(box_loss)/k_fold
  oloss = sum(obj_loss)/k_fold
  closs = sum(cls_loss)/k_fold

  print('Final results using k-fold cross validation')
  print('precision:',p,'recall:',r,'mAP@.5:',mAP5,'mAP@.5_.95:',mAP5_95,'box_loss:',bloss,'obj_loss:',oloss,'cls_loss:',closs)

In [ ]:
def applyGroundTruth(src_annotations, src_images, out_path):

  # read images 
  resolution_of_img = 640

  for f in glob.glob(src_images + "*.jpg"):
    img = cv2.imread(f)

    content = open(src_annotations + str(Path(f).stem) + '.txt', 'r')
    
    # reading the file
    data = content.read()
    data_into_list = data.replace('\n', '').split(' ')
    data_into_list.pop(0)

    i = 0
    while data_into_list:
      x_center = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      y_center = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      width = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      height = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)

      #print(x_center, y_center, width, height)
      img = cv2.circle(img, (int(x_center), int(y_center)), radius = 2, color = (0, 255, 0), thickness = -1)
      xmin = x_center - width/2
      xmax = x_center + width/2
      ymin = y_center - height/2
      ymax = y_center + height/2
      img = cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 1)
      #print(xmin, xmax, ymin, ymax)
      i = i + 1

    cv2.imwrite(out_path + str(Path(f).stem) + ".jpg", img)

    content.close()

In [ ]:
# train with dimensions of bounding boxes: 15x15 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_15_15/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_15_15/"

save_results = "/content/drive/MyDrive/yolov5/results1515/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold1515/"
train_dir = fold_dir+"train/"
data = "/content/drive/MyDrive/yolov5/data/mamoas.yaml"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results1515/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

A saída de streaming foi truncada nas últimas 5000 linhas.

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   128/199     4.63G   0.02244  0.006159         0        39       640: 100% 25/25 [00:05<00:00,  4.19it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:00<00:00,  5.66it/s]
                 all         80        142      0.757      0.394      0.433      0.238

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   129/199     4.63G   0.02417  0.005952         0        33       640: 100% 25/25 [00:05<00:00,  4.29it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:00<00:00,  4.74it/s]
                 all         80        143      0.663      0.441      0.446      0.232

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   130/199     4.63G   0.02247  0.006203         0        37       640: 100% 25/25 [00:05<0

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results1515/trainFolds/"
mean_results(save_results, k_fold)

(0.7791917352792432, 0.45793894694841997, 0.49767118021022855, 0.2790050042421778, 0.061225730925798416, 0.025228505954146385, 0.0)
(0.9211066455552516, 0.8541666666666666, 0.8859915951400257, 0.42568014186334624, 0.04272348806262016, 0.008626628667116165, 0.0)
(0.8553435343930659, 0.6339285714285714, 0.6989910251781297, 0.2408879035123303, 0.05563491955399513, 0.01288691908121109, 0.0)
(0.9372568803052767, 0.7142857142857143, 0.7647732793350983, 0.2412075899000588, 0.05324492231011391, 0.022092996165156364, 0.0)
(0.9538326253604031, 0.5165103374979919, 0.6919810206180267, 0.3235151446663223, 0.04490867257118225, 0.01437834370881319, 0.0)
(0.7962249978004539, 0.66875, 0.6832969305665484, 0.3674677387953321, 0.048494983464479446, 0.021572301164269447, 0.0)
Final results using k-fold cross validation
precision: 0.8738260697822824 recall: 0.6409300394712273 mAP@.5: 0.7037841718413428 mAP@.5_.95: 0.31296058716326125 box_loss: 0.051038786148031555 obj_loss: 0.01746428245678544 cls_loss: 0.0

In [ ]:
# evaluate test dataset
!python val.py --proj /content/drive/MyDrive/yolov5/results1515/val --weights /content/drive/MyDrive/yolov5/results1515/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results1515/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results1515/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.833
!python detect.py --proj /content/drive/MyDrive/yolov5/results1515/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results1515/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results1515/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_15_15/ --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/yolov5/results1515/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results1515/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_15_15/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.833, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results1515/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113

In [ ]:
src_images = "/content/drive/MyDrive/yolov5/results1515/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results1515/detect/exp/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_15_15/"

applyGroundTruth(test_path, src_images, out_path)

In [ ]:
# train with dimensions of bounding boxes: 20x20 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_20_20/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_20_20/"

save_results = "/content/drive/MyDrive/yolov5/results2020/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/"
train_dir = fold_dir+"train/"
data = "/content/drive/MyDrive/yolov5/data/mamoas.yaml"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2020/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/ restaured.
Using 6 folds.
Using fold number: 1
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/fold1
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/train/
Using 6 folds.
Using fold number: 2
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/fold2
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/train/
Using 6 folds.
Using fold number: 3
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/fold3
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/train/
Using 6 folds.
Using fold number: 4
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/fold4
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/train/
Using 6 folds.
Using fold number: 5
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/fold5
/content/drive/MyDrive/dataset_v2/train/folds/datafold2020/train/
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/yolov5/data/mamoas5.yaml, hyp=dat

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results2020/trainFolds/"
mean_results(save_results, k_fold)

(0.8412085239530892, 0.4818181818181818, 0.5778484127946352, 0.35952481451219054, 0.05393116548657417, 0.02741359733045101, 0.0)
(1.0, 0.8854114531197864, 0.9237841015118797, 0.4891467538140805, 0.03368310257792473, 0.009133420884609222, 0.0)
(0.8003087711862455, 0.6160714285714286, 0.674330841474162, 0.3277048598816404, 0.044003311544656754, 0.014103062450885773, 0.0)
(0.9555494017832762, 0.7287091277204271, 0.8034795382225712, 0.33689504868146447, 0.0493767112493515, 0.026118630543351173, 0.0)
(0.982139320807876, 0.539108902826061, 0.6630589645582297, 0.4251148508956383, 0.04538525268435478, 0.024934036657214165, 0.0)
(0.8590953275627048, 0.7142857142857143, 0.7195876287111049, 0.43751329336668093, 0.04112224653363228, 0.02150234580039978, 0.0)
Final results using k-fold cross validation
precision: 0.9063835575488653 recall: 0.6609008013902665 mAP@.5: 0.7270149145454305 mAP@.5_.95: 0.3959832701919492 box_loss: 0.0445836316794157 obj_loss: 0.02053418227781852 cls_loss: 0.0


In [ ]:
!python val.py --proj /content/drive/MyDrive/yolov5/results2020/val --weights /content/drive/MyDrive/yolov5/results2020/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results2020/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results2020/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.529
!python detect.py --proj /content/drive/MyDrive/yolov5/results2020/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results2020/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2020/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_20_20 --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/yolov5/results2020/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2020/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_20_20, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.529, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results2020/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 

In [ ]:
src_images = "/content/drive/MyDrive/yolov5/results2020/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results2020/detect/exp/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_20_20/"

applyGroundTruth(test_path, src_images, out_path)

In [ ]:
#train with dimensions of bounding boxes: 22x22 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_22_22/"
val_path = "/content/drive/MyDrive/dataset_v2/val/data_22_22/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_22_22/"

save_results = "/content/drive/MyDrive/yolov5/results2222/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/"
train_dir = fold_dir+"train/"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2222/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/ restaured.
Using 6 folds.
Using fold number: 1
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/fold1
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/train/
Using 6 folds.
Using fold number: 2
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/fold2
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/train/
Using 6 folds.
Using fold number: 3
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/fold3
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/train/
Using 6 folds.
Using fold number: 4
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/fold4
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/train/
Using 6 folds.
Using fold number: 5
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/fold5
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222/train/
Using 6 folds.
Using fold number: 6
/content/drive/MyDrive/dataset_v2/train/folds/datafold2222

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results2222/trainFolds/"
mean_results(save_results, k_fold)

(0.7768559193844377, 0.5675675675675675, 0.6521248831286408, 0.39212806911547915, 0.05119945481419563, 0.026784265413880348, 0.0)
(0.9777734750107485, 0.9164871606046673, 0.9269545284340208, 0.5523139444015742, 0.02974601648747921, 0.0076932720839977264, 0.0)
(0.8859429049302466, 0.625, 0.7179293230212631, 0.36429739435023567, 0.042972877621650696, 0.012935594655573368, 0.0)
(0.8945825389318776, 0.7264957264957265, 0.7982957508647296, 0.39464344536883583, 0.044464170932769775, 0.02350383810698986, 0.0)
(0.936701965547036, 0.6, 0.7766843497900275, 0.500033686435615, 0.036175910383462906, 0.018051907420158386, 0.0)
(0.7981843096951011, 0.7551020408163265, 0.7568794924531154, 0.48090106972432656, 0.04286335036158562, 0.021789943799376488, 0.0)
Final results using k-fold cross validation
precision: 0.8783401855832412 recall: 0.6984420825807147 mAP@.5: 0.7714780546152995 mAP@.5_.95: 0.44738626823267774 box_loss: 0.041236963433523975 obj_loss: 0.01845980357999603 cls_loss: 0.0


In [ ]:
!python val.py --proj /content/drive/MyDrive/yolov5/results2222/val --weights /content/drive/MyDrive/yolov5/results2222/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results2222/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results2222/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.758
!python detect.py --proj /content/drive/MyDrive/yolov5/results2222/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results2222/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2222/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_22_22 --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/yolov5/results2222/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2222/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_22_22, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.758, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results2222/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 

In [ ]:
src_annotations = "/content/drive/MyDrive/dataset_v2/test/data_22_22/"
src_images = "/content/drive/MyDrive/yolov5/results2222/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results2222/detect/exp/"

applyGroundTruth(src_annotations, src_images, out_path)

In [ ]:
#train with dimensions of bounding boxes: 25x25 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_25_25/"
val_path = "/content/drive/MyDrive/dataset_v2/val/data_25_25/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_25_25/"

save_results = "/content/drive/MyDrive/yolov5/results2525/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold2525/"
train_dir = fold_dir+"train/"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --batch 16 --img 640 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --batch 16 --img 640 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --batch 16 --img 640 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --batch 16 --img 640 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results2525/trainFolds --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

A saída de streaming foi truncada nas últimas 5000 linhas.
    59/199     4.63G   0.02999   0.01051         0        25       640: 100% 25/25 [00:05<00:00,  4.20it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:00<00:00,  5.64it/s]
                 all         80         96      0.965      0.854        0.9      0.513

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    60/199     4.63G   0.02701   0.01083         0        31       640: 100% 25/25 [00:06<00:00,  4.06it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:00<00:00,  4.65it/s]
                 all         80         96      0.977      0.875      0.913      0.482

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    61/199     4.63G   0.03161   0.01098         0        31       640: 100% 25/25 [00:05<00:00,  4.27it/s]
               Class     Images     Labels          P  

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results2525/trainFolds/"
mean_results(save_results, k_fold) 

(0.8138109173555306, 0.5073535239197889, 0.5926139603529607, 0.39829616543572666, 0.0518818162381649, 0.03185923025012016, 0.0)
(0.9333265239964145, 0.8841148372127718, 0.9205199694034332, 0.5589145257297106, 0.030306734144687653, 0.010456402786076069, 0.0)
(0.7731589547679386, 0.6696428571428571, 0.6916422787546281, 0.3705324598157797, 0.04408147931098938, 0.016658179461956024, 0.0)
(0.9368327846550278, 0.7477825867918747, 0.8085293680088086, 0.44057796319145054, 0.039393018931150436, 0.027860650792717934, 0.0)
(0.9848641853425238, 0.6226415094339622, 0.7263698009136232, 0.5079893378908419, 0.03629477322101593, 0.026217559352517128, 0.0)
(0.9007603981887302, 0.7364864864864865, 0.7557389416646805, 0.5015008078539108, 0.03719639778137207, 0.024061432108283043, 0.0)
Final results using k-fold cross validation
precision: 0.8904589607176941 recall: 0.6946703001646236 mAP@.5: 0.7492357198496892 mAP@.5_.95: 0.46296854331957005 box_loss: 0.03985903660456339 obj_loss: 0.022852242458611727 cls

In [ ]:
!python val.py --proj /content/drive/MyDrive/yolov5/results2525/val --weights /content/drive/MyDrive/yolov5/results2525/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results2525/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results2525/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.741
!python detect.py --proj /content/drive/MyDrive/yolov5/results2525/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results2525/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results2525/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_25_25

detect: weights=['/content/drive/MyDrive/yolov5/results2525/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results2525/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_25_25, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.741, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results2525/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu11

In [ ]:
src_annotations = "/content/drive/MyDrive/dataset_v2/test/data_25_25/"
src_images = "/content/drive/MyDrive/yolov5/results2525/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results2525/detect/exp/"

applyGroundTruth(src_annotations, src_images, out_path)

In [ ]:
#train with dimensions of bounding boxes: 30x30 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_30_30/"
val_path = "/content/drive/MyDrive/dataset_v2/val/data_30_30/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_30_30/"

save_results = "/content/drive/MyDrive/yolov5/results3030/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/"
train_dir = fold_dir+"train/"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3030/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/ restaured.
Using 6 folds.
Using fold number: 1
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/fold1
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/train/
Using 6 folds.
Using fold number: 2
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/fold2
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/train/
Using 6 folds.
Using fold number: 3
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/fold3
/content/drive/MyDrive/dataset_v2/train/folds/datafold3030/train/
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/yolov5/data/mamoas3.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/yolov5/

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results3030/trainFolds/"
mean_results(save_results, k_fold) 

(0.8310305401094875, 0.5240798995717029, 0.5922656813167629, 0.41066862729111564, 0.042954202741384506, 0.03198401257395744, 0.0)
(0.9655165397588434, 0.8749817724849531, 0.9180150283669032, 0.5743279659349619, 0.027690844610333443, 0.009945021010935307, 0.0)
(0.9459377748722436, 0.6249014729014729, 0.7131753130308154, 0.44449133628654003, 0.03518618270754814, 0.014926648698747158, 0.0)
(0.9555529111934203, 0.7817700424074926, 0.8432945131598992, 0.48222392430570055, 0.03919604793190956, 0.029759163036942482, 0.0)
(0.9569295613635729, 0.6767676767676768, 0.7773687891397738, 0.565655280224538, 0.031965482980012894, 0.02740948088467121, 0.0)
(0.817427760627107, 0.7251951101888965, 0.7434498096185381, 0.521413442648953, 0.0343693271279335, 0.03602657467126846, 0.0)
Final results using k-fold cross validation
precision: 0.9120658479874457 recall: 0.7012826623870324 mAP@.5: 0.7645948557721155 mAP@.5_.95: 0.49979676278196816 box_loss: 0.03522701468318701 obj_loss: 0.025008483479420345 cls_lo

In [ ]:
!python val.py --proj /content/drive/MyDrive/yolov5/results3030/val --weights /content/drive/MyDrive/yolov5/results3030/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results3030/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results3030/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.436
!python detect.py --proj /content/drive/MyDrive/yolov5/results3030/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results3030/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results3030/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_30_30

detect: weights=['/content/drive/MyDrive/yolov5/results3030/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results3030/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_30_30, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.436, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results3030/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu11

In [ ]:
src_annotations = "/content/drive/MyDrive/dataset_v2/test/data_30_30/"
src_images = "/content/drive/MyDrive/yolov5/results3030/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results3030/detect/exp/"

applyGroundTruth(src_annotations, src_images, out_path)

In [ ]:
#train with dimensions of bounding boxes: 35x35 pixels 

train_path = "/content/drive/MyDrive/dataset_v2/train/data_35_35/"
val_path = "/content/drive/MyDrive/dataset_v2/val/data_35_35/"
test_path = "/content/drive/MyDrive/dataset_v2/test/data_35_35/"

save_results = "/content/drive/MyDrive/yolov5/results3535/trainFolds/"
fold_dir = "/content/drive/MyDrive/dataset_v2/train/folds/datafold3535/"
train_dir = fold_dir+"train/"

fold_paths=splitTrainData(k_fold, train_path, fold_dir)
os.makedirs(train_dir)

for fold in range(k_fold):
  train_dir, fold_val = crossValidation(k_fold, fold, train_dir, fold_paths, fold_dir)
  print(fold_val)
  print(train_dir)
  #createYOLOYaml(train_dir, fold_val, nc, names)
  if(fold == 0):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas1.yaml --weights yolov5s.pt --cache 
  if(fold == 1):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas2.yaml --weights yolov5s.pt --cache 
  if(fold == 2):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas3.yaml --weights yolov5s.pt --cache 
  if(fold == 3):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas4.yaml --weights yolov5s.pt --cache 
  if(fold == 4):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas5.yaml --weights yolov5s.pt --cache 
  if(fold == 5):
    !python train.py --proj /content/drive/MyDrive/yolov5/results3535/trainFolds --img 640 --batch {batch_size} --epochs {n_epochs} --data /content/drive/MyDrive/yolov5/data/mamoas6.yaml --weights yolov5s.pt --cache 
  

/content/drive/MyDrive/dataset_v2/train/folds/datafold3535/ restaured.
Using 6 folds.
Using fold number: 1
/content/drive/MyDrive/dataset_v2/train/folds/datafold3535/fold1
/content/drive/MyDrive/dataset_v2/train/folds/datafold3535/train/
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/yolov5/data/mamoas1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/yolov5/results3535/trainFolds, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 v6.1-157-g70

In [ ]:
#train results
save_results = "/content/drive/MyDrive/yolov5/results3535/trainFolds/"
mean_results(save_results, k_fold) 

(0.912559294291812, 0.4921875, 0.7027433171056453, 0.4459739649445157, 0.04169878736138344, 0.026964323595166206, 0.0)
(0.9885515203501509, 0.90625, 0.9241952100531048, 0.6128509610572793, 0.025484599173069, 0.007712230086326599, 0.0)
(0.8765130386753343, 0.6339285714285714, 0.7053101778683578, 0.4580155608004392, 0.03368343412876129, 0.012646369636058807, 0.0)
(0.9099894873228207, 0.7843830694405407, 0.8384840508045728, 0.5282289699312577, 0.03229746222496033, 0.022566018626093864, 0.0)
(0.9831337678227631, 0.6095238095238096, 0.7724826698640261, 0.5286609585083409, 0.031209206208586693, 0.024450546130537987, 0.0)
(0.8915477284007721, 0.7278911564625851, 0.7651208256729011, 0.49127578700423136, 0.033979982137680054, 0.024785572662949562, 0.0)
Final results using k-fold cross validation
precision: 0.9270491394772754 recall: 0.6923606844759177 mAP@.5: 0.7847227085614347 mAP@.5_.95: 0.5108343670410107 box_loss: 0.0330589118724068 obj_loss: 0.01985417678952217 cls_loss: 0.0


In [ ]:
!python val.py --proj /content/drive/MyDrive/yolov5/results3535/val --weights /content/drive/MyDrive/yolov5/results3535/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp6/weights/best.pt --data /content/drive/MyDrive/yolov5/data/mamoas.yaml --img 640

val: data=/content/drive/MyDrive/yolov5/data/mamoas.yaml, weights=['/content/drive/MyDrive/yolov5/results3535/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results3535/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLO

In [ ]:
confidence = 0.562
!python detect.py --proj /content/drive/MyDrive/yolov5/results3535/detect --conf {confidence} --weights /content/drive/MyDrive/yolov5/results3535/trainFolds/exp/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp3/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp4/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp5/weights/best.pt /content/drive/MyDrive/yolov5/results3535/trainFolds/exp6/weights/best.pt --source /content/drive/MyDrive/dataset_v2/test/data_35_35

detect: weights=['/content/drive/MyDrive/yolov5/results3535/trainFolds/exp/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp2/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp4/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp5/weights/best.pt', '/content/drive/MyDrive/yolov5/results3535/trainFolds/exp6/weights/best.pt'], source=/content/drive/MyDrive/dataset_v2/test/data_35_35, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.562, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results3535/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu11

In [ ]:
src_annotations = "/content/drive/MyDrive/dataset_v2/test/data_35_35/"
src_images = "/content/drive/MyDrive/yolov5/results3535/detect/exp/"
out_path = "/content/drive/MyDrive/yolov5/results3535/detect/exp/"

applyGroundTruth(src_annotations, src_images, out_path)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/results4/train